# 报销分析

本笔记本展示了如何创建使用插件的代理来处理本地收据图像中的差旅费用，生成报销邮件，并通过饼图可视化费用数据。代理会根据任务上下文动态选择功能。

步骤：
1. OCR代理处理本地收据图像并提取差旅费用数据。
2. 邮件代理生成报销邮件。

### 差旅费用场景示例：
假设你是一名员工，为了参加另一座城市的商务会议而出差。你的公司有一项政策，会报销所有合理的差旅相关费用。以下是可能的差旅费用明细：
- 交通费用：
往返于家乡城市和目的地城市的机票费用。  
往返机场的出租车或网约车费用。  
目的地城市的本地交通费用（如公共交通、租车或出租车）。

- 住宿费用：
在会议场地附近的中档商务酒店住宿三晚的费用。

- 餐饮费用：
根据公司每日津贴政策，涵盖早餐、午餐和晚餐的每日餐饮补贴。

- 杂项费用：
机场停车费。  
酒店的网络使用费。  
小费或其他小额服务费用。

- 文档：
你需要提交所有收据（机票、出租车、酒店、餐饮等）以及一份完整的报销报告以申请报销。


## 导入所需库

导入笔记本所需的库和模块。


In [1]:
import os
from dotenv import load_dotenv
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential
from semantic_kernel.kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from openai import AsyncOpenAI
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat


from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies import SequentialSelectionStrategy, DefaultTerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents import ImageContent, TextContent
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, OpenAIChatPromptExecutionSettings

from semantic_kernel.functions import kernel_function, KernelArguments
from pydantic import BaseModel, Field
from typing import List
from azure.ai.inference.models import SystemMessage, UserMessage, TextContentItem, ImageContentItem, ImageUrl, ImageDetailLevel

load_dotenv()

True

In [2]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
   
    client = AsyncOpenAI(
    api_key=os.environ["GITHUB_TOKEN"], base_url="https://models.inference.ai.azure.com/")
    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o-mini",
            async_client=client,
            service_id="open_ai"
        )
    )

    kernel.add_service(
        OpenAIChatCompletion(
            ai_model_id="gpt-4o",
            async_client=client,
            service_id="gpt-4o"
        )
    )

    return kernel

## 定义费用模型

创建一个 Pydantic 模型用于表示单项费用，并创建一个 ExpenseFormatter 类，将用户查询转换为结构化的费用数据。

每项费用将以以下格式表示：
`{'date': '07-Mar-2025', 'description': '飞往目的地的航班', 'amount': 675.99, 'category': '交通'}`


In [3]:
class Expense(BaseModel):
    date: str = Field(..., description="Date of expense in dd-MMM-yyyy format")
    description: str = Field(..., description="Expense description")
    amount: float = Field(..., description="Expense amount")
    category: str = Field(..., description="Expense category (e.g., Transportation, Meals, Accommodation, Miscellaneous)")

class ExpenseFormatter(BaseModel):
    raw_query: str = Field(..., description="Raw query input containing expense details")
    
    def parse_expenses(self) -> List[Expense]:
        """
        Parses the raw query into a list of Expense objects.
        Expected format: "date|description|amount|category" separated by semicolons.
        """
        expense_list = []
        for expense_str in self.raw_query.split(";"):
            if expense_str.strip():
                parts = expense_str.strip().split("|")
                if len(parts) == 4:
                    date, description, amount, category = parts
                    try:
                        expense = Expense(
                            date=date.strip(),
                            description=description.strip(),
                            amount=float(amount.strip()),
                            category=category.strip()
                        )
                        expense_list.append(expense)
                    except ValueError as e:
                        print(f"[LOG] Parse Error: Invalid data in '{expense_str}': {e}")
        return expense_list

## 定义代理 - 生成邮件

创建一个代理类，用于生成提交报销申请的邮件。  
- 该代理使用 `kernel_function` 装饰器定义一个函数，用于生成提交报销申请的邮件。  
- 它会计算费用的总金额，并将详细信息格式化为邮件正文。  


In [4]:
class ExpenseEmailAgent:

    @kernel_function(description="Generate an email to submit an expense claim to the Finance Team")
    async def generate_expense_email(expenses):
        total_amount = sum(expense['amount'] for expense in expenses)
        email_body = "Dear Finance Team,\n\n"
        email_body += "Please find below the details of my expense claim:\n\n"
        for expense in expenses:
            email_body += f"- {expense['description']}: ${expense['amount']}\n"
        email_body += f"\nTotal Amount: ${total_amount}\n\n"
        email_body += "Receipts for all expenses are attached for your reference.\n\n"
        email_body += "Thank you,\n[Your Name]"
        return email_body

# 从收据图片中提取差旅费用的代理

创建一个代理类，用于从收据图片中提取差旅费用。
- 该代理使用 `kernel_function` 装饰器定义一个函数，用于从收据图片中提取差旅费用。
- 使用 OCR（光学字符识别）将收据图片转换为文本，并提取相关信息，例如日期、描述、金额和类别。


In [5]:
class OCRAgentPlugin:
    def __init__(self):
        self.client = ChatCompletionsClient(
            endpoint="https://models.inference.ai.azure.com/",
            credential=AzureKeyCredential(os.environ.get("GITHUB_TOKEN")),
        )
        self.model_name = "gpt-4o"

    @kernel_function(description="Extract structured travel expense data from receipt.jpg using gpt-4o-model")
    def extract_text(self, image_path: str = "receipt.jpg") -> str:
        try:
            image_url_str = str(ImageUrl.load(image_file=image_path, image_format="jpg", detail=ImageDetailLevel.HIGH))

            prompt = (
                "You are an expert OCR assistant specialized in extracting structured data from receipt images. "
                "Analyze the provided receipt image and extract travel-related expense details in the format: "
                "'date|description|amount|category' separated by semicolons. "
                "Follow these rules: "
                "- Date: Convert dates (e.g., '4/4/22') to 'dd-MMM-yyyy' (e.g., '04-Apr-2022'). "
                "- Description: Extract item names (e.g., 'Carlson's Drylawn', 'Peigs transaction Probiotics'). "
                "- Amount: Use numeric values (e.g., '4.50' from '$4.50' or '4.50 dollars'). "
                "- Category: Infer from context (e.g., 'Meals' for food, 'Transportation' for travel, 'Accommodation' for lodging, 'Miscellaneous' otherwise). "
                "Ignore totals, subtotals, or service charges unless they are itemized expenses. "
                "If no expenses are found, return 'No expenses detected'. "
                "Return only the structured data, no additional text."
            )
            response = self.client.complete(
                messages=[
                    SystemMessage(content=prompt),
                    UserMessage(content=[
                        TextContentItem(text="Extract travel expenses from this receipt image."),
                        ImageContentItem(image_url=ImageUrl(url=image_url_str))
                    ])
                ],
                model=self.model_name,
                temperature=0.1,
                max_tokens=2048
            )
            extracted_text = response.choices[0].message.content
            return extracted_text
        except Exception as e:
            error_msg = f"[LOG] OCR Plugin: Error processing image: {str(e)}"
            print(error_msg)
            return error_msg

## 处理费用

定义一个异步函数，通过创建和注册必要的代理，然后调用它们来处理费用。
- 该函数通过加载环境变量、创建必要的代理并将其注册为插件来处理费用。
- 它会创建一个包含两个代理的群聊，并发送提示消息，根据费用数据生成电子邮件和饼图。
- 它会处理在聊天调用过程中发生的任何错误，并确保正确清理代理。


In [6]:
async def process_expenses():
    load_dotenv()
    settings_slm = OpenAIChatPromptExecutionSettings(service_id="gpt-4o")
    settings_llm = OpenAIChatPromptExecutionSettings(service_id="open_ai")  # Fixed typo in service_id
    
    ocr_agent = ChatCompletionAgent(
        kernel=_create_kernel_with_chat_completion("ocrAgent"),
        name="ocr_agent",
        instructions="Extract travel expense data from the receipt image in the prompt using the 'extract_text' function from the 'ocrAgent' plugin. Return the data in the format 'date|description|amount|category' separated by semicolons.",
        arguments=KernelArguments(settings=settings_slm)
    )
    
       
    email_agent = ChatCompletionAgent(
            kernel=_create_kernel_with_chat_completion("expenseEmailAgent"),
            name="email_agent",
            instructions="Take the travel expense data from the previous agent and generate a professional expense claim email using the 'generate_expense_email' function from the 'expenseEmailAgent' plugin, then pass the data forward.",
            arguments=KernelArguments(
                settings=settings_llm)
        )


    kernel = Kernel()

    # Use fixed path to receipt.jpg in the same folder
    image_path = "./receipt.jpg"
    
    # Create a structured message with text and image content for OCR processing
    image_url_str = f"file://{image_path}"
    
    # Using the correct format for multi-modal content
    user_message = ChatMessageContent(
        role=AuthorRole.USER,
        items=[
            TextContent(text="""
            Please extract the raw text from this receipt image, focusing on travel expenses like dates, descriptions, amounts, and categories (e.g., Transportation, Accommodation, Meals, Miscellaneous).
            Then generate a professional expense claim email.
                        """),
            ImageContent.from_image_file(path=image_path)
        ]
    )

    # Register plugins with the kernel
    kernel.add_plugin(OCRAgentPlugin(), plugin_name="ocrAgent")
    kernel.add_plugin(ExpenseEmailAgent(), plugin_name="expenseEmailAgent")

    # Create group chat
    chat = AgentGroupChat(
        agents=[ocr_agent, email_agent],
        selection_strategy=SequentialSelectionStrategy(initial_agent=ocr_agent),
        termination_strategy=DefaultTerminationStrategy(maximum_iterations=1)
    )

    # Add user message with prompt
    await chat.add_chat_message(user_message)
    print(f"# User message added to chat with receipt image")

    async for content in chat.invoke():
        print(f"# Agent - {content.name or '*'}: '{content.content}'")


## 主函数

定义主函数以清空控制台并异步运行 `process_expenses` 函数。


In [9]:
async def main():
    # Clear the console
    os.system('cls' if os.name=='nt' else 'clear')

    # Run the async agent code
    await process_expenses()

await main()

# User message added to chat with receipt image
# Agent - ocr_agent: 'The receipt primarily seems to capture costs for meals and beverages. Below is the extracted travel expense data:

**Travel Expense Data:**  
`2 May '22|Meals at restaurant|75.15|Meals`

---

**Professional Expense Claim Email Draft:**  

**Subject:** Expense Claim for Meals – 2 May 2022  

Dear [Recipient's Name],  

I am submitting an expense claim for a meal incurred during a business-related trip. Below are the details:  

- **Date:** 2 May 2022  
- **Expense Description:** Meals at a restaurant  
- **Amount:** $75.15  
- **Category:** Meals  

Please find the attached receipt for your reference. Kindly process the reimbursement at your earliest convenience. Let me know if you require additional information.  

Thank you for your assistance.  

Best regards,  
[Your Name]  
[Your Contact Information]  

Let me know if you need further revisions or additional details!'



---

**免责声明**：  
本文档使用AI翻译服务[Co-op Translator](https://github.com/Azure/co-op-translator)进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
